In [1]:

import sys
import os
import glob
sys.path.append("../NEAT")
from NEATModels import NEATDynamic, nets
from NEATModels.config import dynamic_config
from NEATUtils import helpers
from NEATUtils.helpers import load_json
from stardist.models import StarDist2D
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
from pathlib import Path

Using TensorFlow backend.


In [2]:
imagedir = '/home/sancere/Kepler/FinalONEATTraining/NEATTest/'
model_dir = '/home/sancere/Kepler/FinalONEATTraining/EverydayneatmodelV2/'
savedir= '/home/sancere/Kepler/FinalONEATTraining/NEATTest/Save/'
model_name = 'divisionm4d74V2'
star_model_name = 'bin2stardist'
division_categories_json = model_dir + 'DivisionCategories.json'
catconfig = load_json(division_categories_json)
division_cord_json = model_dir + 'DivisionCord.json'
cordconfig = load_json(division_cord_json)
model = NEATDynamic(None, model_dir , model_name,catconfig, cordconfig)
starmodel = StarDist2D(config = None, name = star_model_name, basedir = model_dir)
Path(savedir).mkdir(exist_ok=True)
n_tiles = (1,1)
event_threshold = 0.9
iou_threshold = 0.6
yolo_v2 = True

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Loading network weights from 'weights_last.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.65, nms_thresh=0.075.


# In the code block below compute the markers and make a dictionary for each image

In [3]:
Raw_path = os.path.join(imagedir, '*tif')
X = glob.glob(Raw_path)
marker_dict = {}
for imagename in X:
     markers, markers_tree, density_location =  model.markers(imagename, starmodel, n_tiles = n_tiles)
     
     marker_dict[imagename] = [markers, markers_tree, density_location]
     

  0%|          | 0/20 [00:00<?, ?it/s]

Obtaining Markers



 40%|████      | 8/20 [00:00<00:00, 75.32it/s]

Creating Dictionary of marker location for fast search


 25%|██▌       | 5/20 [00:00<00:00, 41.25it/s]

Computing density of each marker


100%|██████████| 20/20 [00:00<00:00, 37.73it/s]


# In the cell below use the computed dictionary and apply division model detection

In [4]:
Raw_path = os.path.join(imagedir, '*tif')
X = glob.glob(Raw_path)
for imagename in X:
     
     markers, markers_tree, density_location = marker_dict[imagename]
     model.predict(imagename,markers, markers_tree, density_location, savedir, n_tiles = n_tiles, event_threshold = event_threshold, iou_threshold = iou_threshold)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  0%|          | 0/10 [00:00<?, ?it/s]

Detecting negative events to exclude from prediction


  0%|          | 0/20 [00:00<?, ?it/s]

Detecting event locations


  0%|          | 0/20 [00:00<?, ?it/s]

(1, 1, 1, 9)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:

model.showNapari(imagedir, savedir, yolo_v2)
